# 利用 Python 进行数据分析

## 第 6 章 数据加载、存储与文件格式

### 读写文本格式的数据

| 函数       | 说明          |
| ----       | -----------------------------------------------------------------------------------  |
| read_csv   | 从文件、URL、文件型对象中加载带分隔符的数据。默认分隔符为 逗号                       |
| read_fwf   | 读取定宽列格式数据（也就是说，没有分隔符）                                           |
| read_clipboard | 读取剪贴板中的数据，可以看做 read_table 的剪贴板版。在将网页转换为表格时很有用   |
| read_excel | 从 Excel XLS 或 XLSX file 读取表格数据           |
| read_hdf | 读取 pandas 写的 HDF5 文件                         |
| read_html | 读取 HTML 文档中的所有表格                         |
| read_json | 读取 JSON(JavaScript Object Notation)字符串中的数据              |
| read_msgpack | 二进制格式编码的 pandas 数据                |
| read_pickle | 读取 Python pickle 格式中存储的任意对象      |
| read_sas | 读取存储于 SAS 系统自定义存储格式的 SAS 数据集  |
| read_sql | （使用 SQLAlchemy）读取 SQL 查询结果为 pandas 的 DataFrame |
| read_stata | 读取 Stata 文件格式的数据集                         |
| read_feather | 读取 Feather 二进制文件格式                         |

In [62]:
!type ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [64]:
import pandas as pd

df = pd.read_csv('ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [2]:
import pandas as pd
pd.read_csv('ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [67]:
# 没有标题行的文件 读入方式
!type ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [68]:
# ① 让 pandas 为其分配默认的列名
import pandas as pd
pd.read_csv('ex2.csv', header=None)


,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [69]:
# ②自定义列名
import pandas as pd
pd.read_csv('ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [70]:
# 希望将message列做成DataFrame的索引。你可以明确表示要将该列放到索引4的位置上，
# 也可以通过index_col参数指定"message"：
import pandas as pd
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [72]:
# 如果希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可：
!type csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [73]:
import pandas as pd
parsed = pd.read_csv('csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [91]:
# 有些表格可能不是用固定的分隔符去分隔字段的（比如空白符或其它模式）
list(open('ex3.txt'))

['            A         B         C \n',
 'aaa -0.264438 -1.026059 -0.619500 \n',
 'bbb 0.927272 0.302904 -0.032399 \n',
 'ccc -0.264273 -0.386314 -0.217601 \n',
 'ddd -0.871858 -0.348382 1.100491 ']

In [92]:
import pandas as pd
result = pd.read_csv('ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [94]:
# 用skiprows跳过文件的第一行、第三行和第四行
import pandas as pd
!type ex4.csv

pd.read_csv('ex4.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [95]:
# 缺失值处理是文件解析任务中的一个重要组成部分。缺失数据经常是要么没有（空字符串），
# 要么用某个标记值表示。默认情况下，pandas会用一组经常出现的标记值进行识别，比如NA及NULL：

!type ex5.csv

import pandas as pd
result = pd.read_csv('ex5.csv')
result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [96]:
!type ex5.csv

import pandas as pd
result = pd.read_csv('ex5.csv')
pd.isnull(result)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [97]:
!type ex5.csv
# na_values可以用一个列表或集合的字符串表示缺失值：
import pandas as pd
result = pd.read_csv('ex5.csv', na_values=['NULL'])
result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [99]:
# 字典的各列可以使用不同的NA标记值：
!type ex5.csv

import pandas as pd
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('ex5.csv', na_values=sentinels)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 逐块读取文本文件

In [3]:
import pandas as pd
pd.options.display.max_rows = 10
result = pd.read_csv('ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [4]:
# 如果只想读取几行（避免读取整个文件），通过 nrows 进行指定即可
import pandas as pd

pd.read_csv('ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [5]:
# 要逐块读取文件，可以指定chunksize（行数）：
import pandas as pd

chunker = pd.read_csv('ex6.csv', chunksize=1000)
chunker

In [6]:
import pandas as pd
from pandas import Series

chunker = pd.read_csv('ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 将数据写出到文本格式

In [7]:
import pandas as pd
data = pd.read_csv('ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [9]:
# 利用DataFrame的to_csv方法，我们可以将数据写到一个以逗号分隔的文件中
import pandas as pd
data.to_csv('out.csv')
!type out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo
